In [1]:
import sys
import os

parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(parent_dir)

In [2]:
import numpy as np
import calendar
from helpers import *
from thermal_optimal_path.lattice import partition_function
from thermal_optimal_path.statistics import average_path
from strategies import basic_strategy
import matplotlib.pyplot as plt
from scipy.stats import norm


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/home/elantonino/Anaconda3/envs/gt/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/elantonino/Anaconda3/envs/gt/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/home/elantonino/Anaconda3/envs/gt/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 739, in start
    self

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/home/elantonino/Anaconda3/envs/gt/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/elantonino/Anaconda3/envs/gt/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/home/elantonino/Anaconda3/envs/gt/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 739, in start
    self

ImportError: 
A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.



In [3]:
year = 2024
month = 7
temperature = 0.1
minute = 40

days_in_month = calendar.monthrange(year, month)[1]

In [4]:
daily_results = {}

for day in range(1, days_in_month + 1):
    day_str = f"{day:02d}"  
    print(f"Processing day: {year}-{month:02d}-{day_str}")
    
    btc_path = f'/home/elantonino/Big data project/btc_parquet_data/{year}/{month:02d}/BTCUSDT-1s-{year}-{month:02d}-{day_str}.parquet'
    eth_path = f'/home/elantonino/Big data project/eth_parquet_data/{year}/{month:02d}/ETHUSDT-1s-{year}-{month:02d}-{day_str}.parquet'

    try:
        btc_data = load_parquet(btc_path)
        eth_data = load_parquet(eth_path)
    except FileNotFoundError:
        print(f"Data for {year}-{month:02d}-{day_str} not found, skipping...")
        continue

    btc_prices = btc_data['Open']
    eth_prices = eth_data['Open']
    btc_prices_std = standardize(btc_prices)
    eth_prices_std = standardize(eth_prices)

    btc_returns = btc_prices.pct_change().rolling(window=5).mean()[minute:]
    eth_returns = eth_prices.pct_change().rolling(window=5).mean()[minute:]

    df_avg = []
    weights = np.exp(np.linspace(0, 2, minute))  
    weights /= weights.sum()
    total = len(btc_prices_std)
    for i in range(total - minute):
        end_time = i + minute
        btc = btc_prices_std[i:end_time]
        eth = eth_prices_std[i:end_time]

        g = partition_function(btc, eth, temperature)
        avg_path = average_path(g)[::2]
        weighted_avg = np.dot(avg_path, weights)
        df_avg.append(weighted_avg)

    strategy_result = basic_strategy(
        btc_prices[minute:], 
        btc_returns, 
        eth_prices[minute:], 
        eth_returns, 
        df_avg
    )

    daily_results[day_str] = {
        "df_avg": df_avg,
        "btc_returns": btc_returns,
        "eth_returns": eth_returns,
        "strategy_result": strategy_result
    }

print("Finished processing all days.")


Processing day: 2024-07-01
Total money after strategy: Base amount + 0.0005185709473746192
Processing day: 2024-07-02
Total money after strategy: Base amount + 0.00029155768874411514
Processing day: 2024-07-03
Total money after strategy: Base amount + 0.0002613864534108945
Processing day: 2024-07-04
Total money after strategy: Base amount + -0.0001661169864545231
Processing day: 2024-07-05
Total money after strategy: Base amount + -0.0006346929539805535
Processing day: 2024-07-06
Total money after strategy: Base amount + 0.0003920871184581731
Processing day: 2024-07-07
Total money after strategy: Base amount + 0.0007016783001616567
Processing day: 2024-07-08
Total money after strategy: Base amount + 0.0031626167793078475
Processing day: 2024-07-09
Total money after strategy: Base amount + 0.001436797386969113
Processing day: 2024-07-10
Total money after strategy: Base amount + -0.00025197738479820764
Processing day: 2024-07-11
Total money after strategy: Base amount + 0.000733780223603

KeyboardInterrupt: 